# Simularium Conversion Tutorial : MEDYAN Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio.medyan import MedyanConverter, MedyanData, MedyanAgentInfo

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium JSON file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium `MedyanConverter` consumes discrete cell data from MEDYAN. 

The converter requires a `MedyanData` object as a parameter.

`MedyanData` contains the following:          
* **box_size** : *np.ndarray (shape = \[3\])*
    * A numpy ndarray containing the XYZ dimensions of the cubic simulation bounding volume          
* **path_to_snapshot** : *string*
    * A string path to the MEDYAN snapshot.traj output file          
* **filament_agent_info** : *Dict\[int, MedyanAgentInfo\] (optional)*
    * A dict mapping MEDYAN type ID for filaments to info (name, radius) for filament agents          
* **linker_agent_info** : *Dict\[int, MedyanAgentInfo\] (optional)*
    * A dict mapping MEDYAN type ID for linkers to info (name, radius) for linker agents          
* **motor_agent_info** : *Dict\[int, MedyanAgentInfo\] (optional)*
    * A dict mapping MEDYAN type ID for motors to info (name, radius) for motor agents          
* **draw_fiber_points** : *bool (optional)*
    * in addition to drawing a line for each fiber, also draw spheres at every other point along it?
    * Default: False
* **scale_factor** : *float (optional)*
    * A multiplier for the MEDYAN scene, use if visualization is too large or small
    * Default: 1.0          
* **plots** : *List\[Dict\[str, Any\]\] (optional)*
    * An object containing plot data already in Simularium format

In [2]:
box_size = 1000.0

example_data = MedyanData(
    box_size=np.array([1000.0, 1000.0, 500.0]),
    path_to_snapshot="../simulariumio/tests/data/medyan/snapshot.traj",
    filament_agent_info={
        0: MedyanAgentInfo(
            name="Actin",
            radius=1.0,
        ),
    },
    linker_agent_info={
        0: MedyanAgentInfo(
            name="XlinkA",
            radius=1.0,
        ),
        1: MedyanAgentInfo(
            name="XlinkB",
            radius=1.0,
        ),
        2: MedyanAgentInfo(
            name="XlinkC",
            radius=1.0,
        ),
    },
    motor_agent_info={
        1: MedyanAgentInfo(
            name="Motor",
            radius=2.0,
        ),
    }
)

## Convert and save as .simularium JSON file

Once your data is shaped like in the `example_data` object, you can use the converter to generate the file at the given path:

In [3]:
MedyanConverter(example_data).write_JSON("medyan_example")

Reading MEDYAN Data -------------
Writing JSON -------------
saved to medyan_example.simularium


## Visualize in the Simularium viewer

In a supported web-browser (Firefox or Chrome), navigate to https://simularium.allencell.org/ and import your file into the view.